# Method 3: CNN with Multi-Head Attention (Fixed)

**Goal:** Train a high-accuracy music genre classifier using a Convolutional Neural Network (CNN) enhanced with Multi-Head Attention.

**Key Innovations:**
- **Data Augmentation**: Increases training data size using time stretching, pitch shifting, and noise injection to combat overfitting on the small GTZAN dataset.
- **Segment-based Processing**: Splits each song into 10 temporal segments. A shared CNN processes each segment independently (TimeDistributed).
- **Multi-Head Attention**: Aggregates information from all segments, allowing the model to weigh the importance of different parts of the song dynamically.
- **Label Smoothing**: Regularization technique to prevent the model from becoming too confident in its predictions.

### Imports & Configuration
**Description:** Import necessary libraries and set hyperparameters. 
- `N_MELS = 64`: Number of Mel bands (smaller than Method 1/2 to reduce dimensionality for attention).
- `NUM_SEGMENTS = 10`: Number of chunks each 30s audio file is split into.

**Outputs:**
- Libraries loaded
- TensorFlow version printed

In [1]:
import numpy as np
import librosa
import os
import random
import warnings
warnings.filterwarnings('ignore')

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, Model, regularizers
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.utils import to_categorical

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

import matplotlib.pyplot as plt
import seaborn as sns

# Seeds for reproducibility
SEED = 42
np.random.seed(SEED)
tf.random.set_seed(SEED)
random.seed(SEED)

# Config
SAMPLE_RATE = 22050
DURATION = 30
N_MELS = 64
N_FFT = 2048
HOP_LENGTH = 512
NUM_SEGMENTS = 10
NUM_CLASSES = 10

GENRES = ['blues', 'classical', 'country', 'disco', 'hiphop',
          'jazz', 'metal', 'pop', 'reggae', 'rock']

DATA_PATH = '../data/gtzan/genres_original'
OUTPUT_DIR = '../models'
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)

print(f'TensorFlow: {tf.__version__}')

## 1. Data Augmentation
**Description:** Define transformations to artificially expand the dataset.
- **Time Stretch**: Changes speed without changing pitch.
- **Pitch Shift**: Changes pitch without changing speed.
- **Noise Injection**: Adds random Gaussian noise.

**Returns:** 
- List containing original audio plus augmented versions.

In [2]:
def augment_audio(y, sr):
    """Apply audio augmentations to increase training data."""
    augmented = [y]  # Original
    
    # Time stretch
    rate = np.random.uniform(0.9, 1.1)
    y_stretch = librosa.effects.time_stretch(y, rate=rate)
    # Fix length after stretching
    if len(y_stretch) > len(y):
        y_stretch = y_stretch[:len(y)]
    else:
        y_stretch = np.pad(y_stretch, (0, len(y) - len(y_stretch)))
    augmented.append(y_stretch)
    
    # Pitch shift
    n_steps = np.random.randint(-2, 3)
    if n_steps != 0:
        y_pitch = librosa.effects.pitch_shift(y, sr=sr, n_steps=n_steps)
        augmented.append(y_pitch)
    
    # Add noise
    noise = np.random.normal(0, 0.003, len(y))
    augmented.append(y + noise)
    
    return augmented

print('Augmentation functions defined')

## 2. Feature Extraction
**Description:** 
1. Compute Mel Spectrogram for the entire 30s clip.
2. Split the spectrogram into `NUM_SEGMENTS` equal parts.

**Inputs:**
- Raw audio array

**Outputs:**
- 3D array: `(num_segments, n_mels, time_bins_per_segment)`

In [3]:
def extract_segmented_melspec(y, sr, num_segments=NUM_SEGMENTS):
    """Extract mel-spectrogram and segment into parts."""
    # Full mel-spectrogram
    mel = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=N_MELS, 
                                          n_fft=N_FFT, hop_length=HOP_LENGTH)
    mel_db = librosa.power_to_db(mel, ref=np.max)
    
    # Segment into equal parts
    total_frames = mel_db.shape[1]
    segment_len = total_frames // num_segments
    
    segments = []
    for i in range(num_segments):
        start = i * segment_len
        end = start + segment_len
        seg = mel_db[:, start:end]
        segments.append(seg)
    
    return np.array(segments)  # (10, 64, segment_len)

print('Feature extraction defined')

## 3. Load & Process Dataset
**Description:** Iterate through genres, load audio, apply augmentation, and extract segmental features for every file. This significantly increases the effective dataset size (approx 4x).

**Outputs:**
- `X`: Features array
- `y`: Labels array

In [4]:
def load_dataset(data_path, augment=True):
    """Load GTZAN with optional augmentation."""
    X, y = [], []
    
    for genre in GENRES:
        genre_path = os.path.join(data_path, genre)
        if not os.path.exists(genre_path):
            continue
        
        files = sorted([f for f in os.listdir(genre_path) if f.endswith('.wav')])
        if 'jazz.00054.wav' in files:
             files.remove('jazz.00054.wav') # Corrupt file

        print(f'{genre}: {len(files)} files')
        
        for f in files:
            try:
                filepath = os.path.join(genre_path, f)
                audio, sr = librosa.load(filepath, sr=SAMPLE_RATE, duration=DURATION)
                
                # Pad/trim to exact length
                target_len = SAMPLE_RATE * DURATION
                if len(audio) < target_len:
                    audio = np.pad(audio, (0, target_len - len(audio)))
                else:
                    audio = audio[:target_len]
                
                # Get audio versions (original + augmented)
                if augment:
                    audio_versions = augment_audio(audio, sr)
                else:
                    audio_versions = [audio]
                
                for av in audio_versions:
                    features = extract_segmented_melspec(av, sr)
                    X.append(features)
                    y.append(genre)
                    
            except Exception as e:
                print(f'Error: {f}: {e}')
    
    return np.array(X), np.array(y)

print('Loading dataset with augmentation...')
X, y = load_dataset(DATA_PATH, augment=True)
print(f'\nTotal samples: {len(X)}')
print(f'Shape: {X.shape}')

## 4. Split and Normalize
**Description:** 
- Encode string labels to integers.
- stratified Train/Val/Test split.
- **Normalization:** Standard scaling `(X - mean) / std` using training set statistics.
- **Label Smoothing:** Applied to `to_categorical` (though standard keras `to_categorical` doesn't support smoothing directly, so we might implement it in the loss or just use soft targets if manually implemented. **Note:** In this code block, `to_categorical` is used simply. Label smoothing will actually be handled by the loss function if `label_smoothing` argument is used in `categorical_crossentropy` or if we manually smoothed labels—wait, checking the code—the code uses `loss='categorical_crossentropy'` later. If `label_smoothing` isn't in compile, it's just standard One-Hot. Let's start with standard execution).

**Outputs:**
- Normalized `X_train`, `X_val`, `X_test`
- `mean`, `std` (to be saved later)

In [5]:
# Encode labels
le = LabelEncoder()
y_enc = le.fit_transform(y)

# Split: 80/10/10
X_temp, X_test, y_temp, y_test = train_test_split(
    X, y_enc, test_size=0.1, stratify=y_enc, random_state=SEED)
X_train, X_val, y_train, y_val = train_test_split(
    X_temp, y_temp, test_size=0.111, stratify=y_temp, random_state=SEED)

# Normalize
mean = X_train.mean()
std = X_train.std()
X_train = (X_train - mean) / (std + 1e-8)
X_val = (X_val - mean) / (std + 1e-8)
X_test = (X_test - mean) / (std + 1e-8)

# Add channel dim -> (batch, segments, mels, time, 1)
X_train = X_train[..., np.newaxis]
X_val = X_val[..., np.newaxis]
X_test = X_test[..., np.newaxis]

# One-hot
y_train_cat = to_categorical(y_train, NUM_CLASSES)
y_val_cat = to_categorical(y_val, NUM_CLASSES)
y_test_cat = to_categorical(y_test, NUM_CLASSES)

print(f'Train: {X_train.shape}, Val: {X_val.shape}, Test: {X_test.shape}')

## 5. Custom Multi-Head Attention
**Description:** A custom Keras layer implementing Scaled Dot-Product Attention with multiple heads. 
- `split_heads`: Divides the query/key/value vectors into multiple heads.
- `call`: Computes attention scores and weighted sum of values.

**Why?** Allows the model to focus on the most characteristic segments of a song (e.g., the chorus or a solo) rather than treating all parts equally.

In [6]:
class MultiHeadAttention(layers.Layer):
    """Simplified Multi-Head Attention for small datasets."""
    
    def __init__(self, d_model=64, num_heads=4, dropout_rate=0.1, **kwargs):
        super().__init__(**kwargs)
        self.d_model = d_model
        self.num_heads = num_heads
        self.depth = d_model // num_heads
        self.dropout_rate = dropout_rate
        
        self.wq = layers.Dense(d_model)
        self.wk = layers.Dense(d_model)
        self.wv = layers.Dense(d_model)
        self.dense = layers.Dense(d_model)
        self.dropout = layers.Dropout(dropout_rate)
    
    def split_heads(self, x, batch_size):
        x = tf.reshape(x, (batch_size, -1, self.num_heads, self.depth))
        return tf.transpose(x, perm=[0, 2, 1, 3])
    
    def call(self, inputs, training=False):
        batch_size = tf.shape(inputs)[0]
        
        q = self.split_heads(self.wq(inputs), batch_size)
        k = self.split_heads(self.wk(inputs), batch_size)
        v = self.split_heads(self.wv(inputs), batch_size)
        
        scale = tf.math.sqrt(tf.cast(self.depth, tf.float32))
        attention = tf.nn.softmax(tf.matmul(q, k, transpose_b=True) / scale, axis=-1)
        attention = self.dropout(attention, training=training)
        
        out = tf.matmul(attention, v)
        out = tf.transpose(out, perm=[0, 2, 1, 3])
        out = tf.reshape(out, (batch_size, -1, self.d_model))
        
        return self.dense(out)

print('MultiHeadAttention defined: 4 heads, 64 dimensions')

## 6. Build Model Architecture
**Structure:**
1.  **Input:** `(10, 64, 129, 1)` - 10 segments of spectrogram images.
2.  **TimeDistributed CNN:** A simpler CNN (2 blocks) is applied to *each segment independently*. This extracts a feature vector for every segment.
3.  **Multi-Head Attention:** Processes the sequence of 10 feature vectors.
4.  **Global Pooling:** Averages the attention outputs over time.
5.  **Classifier:** Dense layer with Dropout and Softmax.

**Outputs:**
- Compiled Keras model summary.

In [7]:
def build_segment_cnn(input_shape):
    """CNN for each segment - smaller architecture."""
    inputs = layers.Input(shape=input_shape)
    
    x = layers.Conv2D(32, (3,3), padding='same')(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    x = layers.MaxPooling2D((2,2))(x)
    
    x = layers.Conv2D(64, (3,3), padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    x = layers.MaxPooling2D((2,2))(x)
    
    x = layers.GlobalAveragePooling2D()(x)
    
    return Model(inputs, x, name='segment_cnn')

def build_model(segment_shape):
    """Full model with multi-head attention."""
    inputs = layers.Input(shape=(NUM_SEGMENTS,) + segment_shape)
    
    # CNN per segment
    cnn = build_segment_cnn(segment_shape)
    x = layers.TimeDistributed(cnn)(inputs)  # (batch, 10, 64)
    
    # Multi-head attention
    x = MultiHeadAttention(d_model=64, num_heads=4, dropout_rate=0.2)(x)
    
    # Pool over time
    x = layers.GlobalAveragePooling1D()(x)
    
    # Classifier
    x = layers.Dense(64, activation='relu')(x)
    x = layers.Dropout(0.5)(x)
    outputs = layers.Dense(NUM_CLASSES, activation='softmax')(x)
    
    model = Model(inputs, outputs)
    
    # Use label smoothing in loss if possible, or manual. Here using standard cat_cross.
    # Note: To apply label smoothing in TF < 2.2, one would implement a custom loss.
    # For simplicity, we use standard loss as placeholder or built-in if version supports.
    # TF 2.x supports label_smoothing argument in Loss class.
    loss = tf.keras.losses.CategoricalCrossentropy(label_smoothing=0.1)
    
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=0.0003),
        loss=loss,
        metrics=['accuracy']
    )
    
    return model

# Build
segment_shape = X_train.shape[2:]  # (64, seg_len, 1)
model = build_model(segment_shape)
model.summary()

## 7. Training
**Description:** Train the model with careful regularization mechanics:
- **EarlyStopping:** Stops if validation accuracy doesn't improve for 20 epochs.
- **ReduceLROnPlateau:** Lowers learning rate if progress stalls.
- **Epochs:** 100 (stopped early usually).

**Outputs:**
- Training logs

In [8]:
callbacks = [
    EarlyStopping(monitor='val_accuracy', patience=20, restore_best_weights=True),
    ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=7, min_lr=1e-6)
]

history = model.fit(
    X_train, y_train_cat,
    validation_data=(X_val, y_val_cat),
    epochs=100,
    batch_size=32,
    callbacks=callbacks,
    verbose=1
)

## 8. Evaluation
**Description:** Assess model performance on the unseen test set.

**Outputs:**
- Accuracy/Loss Plots
- Test Accuracy Score
- Classification Report (Precision, Recall, F1-Score per genre)

In [9]:
# Plot accuracy/loss
fig, axes = plt.subplots(1, 2, figsize=(12, 4))
axes[0].plot(history.history['accuracy'], label='Train')
axes[0].plot(history.history['val_accuracy'], label='Val')
axes[0].set_title('Accuracy')
axes[0].legend()

axes[1].plot(history.history['loss'], label='Train')
axes[1].plot(history.history['val_loss'], label='Val')
axes[1].set_title('Loss')
axes[1].legend()
plt.show()

# Test Eval
test_loss, test_acc = model.evaluate(X_test, y_test_cat, verbose=0)
print(f"\nTest Accuracy: {test_acc*100:.2f}%")

y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)

print("\nClassification Report:")
print(classification_report(y_test, y_pred_classes, target_names=le.classes_))

**Description:** Assess model performance on the unseen test set.

**Outputs:**
- Accuracy/Loss Plots
- Test Accuracy Score
- Classification Report (Precision, Recall, F1-Score per genre)

In [10]:
model.save(f'{OUTPUT_DIR}/method3_attention.keras')
np.savez(f'{OUTPUT_DIR}/method3_scalars.npz', mean=mean, std=std)
print("Model and scalars saved.")